In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns

df = pd.read_csv("C:/Users/ajays/Downloads/consolidated_dataset_distance.csv")
df.head()

,CustomerID,CustomerAreaCode,CustomerPostalCode,CustomerPostalCodeLatitude,CustomerPostalCodeLongitude,CustomerDMACode,CustomerDMADescription,NCAACustomerRecordCreated,BracketEntryId,BracketEntryCreatedDate,...,InstitutionNCAAMemberSinceDate_NationalChampion,RegularSeasonWins_NationalChampion,RegularSeasonLosses_NationalChampion,RegularSeasonAverageAttendance_NationalChampion,RegularSeasonAverageScore_NationalChampion,Distance_RegionWinner_East,Distance_RegionWinner_West,Distance_RegionWinner_South,Distance_RegionWinner_Midwest,Distance_NationalChampion
0,47028,NaN,36093,32.5622,-86.0994,698.0,MONTGOMERY (SELMA),12/25/21,1723503,3/19/24 10:27,...,09-01-2009,24,8,16065.47,79.47,16.652700,24.853585,9.314636,4.026861,4.026861
1,3511,616.0,49464,42.8256,-86.0104,563.0,GRAND RAPIDS - KALMZOO - B. CRK,04-02-2021,963479,3/18/24 10:16,...,09-01-2010,31,3,14017.88,81.47,13.797944,15.832122,5.017906,10.013960,13.797944
2,58445,703.0,22210,38.8808,-77.1129,511.0,"WASHINGTON, DC (HAGRSTWN)",04-02-2021,810038,3/18/24 0:21,...,09-01-2014,29,4,13329.06,83.39,5.676550,21.309908,20.402680,9.922936,9.922936
3,28833,NaN,78218,29.4969,-98.4032,641.0,SAN ANTONIO,11/16/23,3384825,3/21/24 10:28,...,09-01-1941,22,14,12147.11,76.36,13.401145,25.131344,20.706588,15.858283,20.706588
4,37899,NaN,14212,42.8946,-78.8245,514.0,BUFFALO,3/16/22,2828017,3/20/24 20:14,...,09-01-2006,27,7,15767.47,81.47,12.263656,6.986502,9.104606,17.200644,6.986502


In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# ================================
# Data Preparation (All Stages)
# ================================
# Convert team ID columns to integer type
df['RegionWinner_East'] = df['RegionWinner_East'].astype(int)
df['RegionWinner_West'] = df['RegionWinner_West'].astype(int)
df['RegionWinner_South'] = df['RegionWinner_South'].astype(int)
df['RegionWinner_Midwest'] = df['RegionWinner_Midwest'].astype(int)
df['SemifinalWinner_East_West'] = df['SemifinalWinner_East_West'].astype(int)
df['SemifinalWinner_South_Midwest'] = df['SemifinalWinner_South_Midwest'].astype(int)
df['NationalChampion'] = df['NationalChampion'].astype(int)

# Define a common hyperparameter grid (you can adjust these values)
param_grid = {
    'learning_rate': [0.08, 0.1, 0.12],
    'max_depth': [5, 6],
    'n_estimators': [50, 75, 100],
    'subsample': [0.9, 1],
    'colsample_bytree': [0.8, 1]
}

# ========================================
# Part 1: Semifinal 1 (East vs. West)
# ========================================
# Create binary target: 0 if East wins, 1 if West wins.
df['target_semi1'] = df.apply(
    lambda row: 0 if row['SemifinalWinner_East_West'] == row['RegionWinner_East']
                else (1 if row['SemifinalWinner_East_West'] == row['RegionWinner_West'] else -1),
    axis=1
)
df_semi1 = df[df['target_semi1'] != -1].copy().reset_index(drop=True)

# Define features for Semifinal 1 (adjust as needed)
feature_cols_semi1 = [
    'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
    'InstitutionCity_RegionWinner_East', 'InstitutionState_RegionWinner_East', 'InstitutionConference_RegionWinner_East',
    'InstitutionEnrollment_Male_RegionWinner_East', 'InstitutionEnrollment_Female_RegionWinner_East', 'InstitutionEnrollment_Total_RegionWinner_East',
    'RegularSeasonWins_RegionWinner_East', 'RegularSeasonLosses_RegionWinner_East', 'RegularSeasonAverageAttendance_RegionWinner_East', 'RegularSeasonAverageScore_RegionWinner_East',
    'InstitutionCity_RegionWinner_West', 'InstitutionState_RegionWinner_West', 'InstitutionConference_RegionWinner_West',
    'InstitutionEnrollment_Male_RegionWinner_West', 'InstitutionEnrollment_Female_RegionWinner_West', 'InstitutionEnrollment_Total_RegionWinner_West',
    'RegularSeasonWins_RegionWinner_West', 'RegularSeasonLosses_RegionWinner_West', 'RegularSeasonAverageAttendance_RegionWinner_West', 'RegularSeasonAverageScore_RegionWinner_West',
    'InstitutionCity_RegionWinner_South', 'InstitutionState_RegionWinner_South', 'InstitutionConference_RegionWinner_South',
    'InstitutionEnrollment_Male_RegionWinner_South', 'InstitutionEnrollment_Female_RegionWinner_South', 'InstitutionEnrollment_Total_RegionWinner_South',
    'RegularSeasonWins_RegionWinner_South', 'RegularSeasonLosses_RegionWinner_South', 'RegularSeasonAverageAttendance_RegionWinner_South', 'RegularSeasonAverageScore_RegionWinner_South',
    'InstitutionCity_RegionWinner_Midwest', 'InstitutionState_RegionWinner_Midwest', 'InstitutionConference_RegionWinner_Midwest',
    'InstitutionEnrollment_Male_RegionWinner_Midwest', 'InstitutionEnrollment_Female_RegionWinner_Midwest', 'InstitutionEnrollment_Total_RegionWinner_Midwest',
    'RegularSeasonWins_RegionWinner_Midwest', 'RegularSeasonLosses_RegionWinner_Midwest', 'RegularSeasonAverageAttendance_RegionWinner_Midwest', 'RegularSeasonAverageScore_RegionWinner_Midwest',
    'Distance_RegionWinner_East', 'Distance_RegionWinner_West', 'Distance_RegionWinner_South', 'Distance_RegionWinner_Midwest'
]

X_semi1 = df_semi1[feature_cols_semi1]
y_semi1 = df_semi1['target_semi1']

# List categorical columns for one-hot encoding
categorical_cols_semi1 = [
    'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
    'InstitutionCity_RegionWinner_East', 'InstitutionState_RegionWinner_East', 'InstitutionConference_RegionWinner_East',
    'InstitutionCity_RegionWinner_West', 'InstitutionState_RegionWinner_West', 'InstitutionConference_RegionWinner_West',
    'InstitutionCity_RegionWinner_South', 'InstitutionState_RegionWinner_South', 'InstitutionConference_RegionWinner_South',
    'InstitutionCity_RegionWinner_Midwest', 'InstitutionState_RegionWinner_Midwest', 'InstitutionConference_RegionWinner_Midwest'
]
X_semi1 = pd.get_dummies(X_semi1, columns=categorical_cols_semi1, drop_first=True)

# Split data
X_train_semi1, X_test_semi1, y_train_semi1, y_test_semi1 = train_test_split(X_semi1, y_semi1, test_size=0.30, random_state=42)

# Set up GridSearchCV for Semifinal 1
xgb_model = XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)
grid_search_semi1 = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search_semi1.fit(X_train_semi1, y_train_semi1)

best_model_semi1 = grid_search_semi1.best_estimator_
print("Semifinal 1 best parameters:", grid_search_semi1.best_params_)

y_pred_semi1 = best_model_semi1.predict(X_test_semi1)
print("Semifinal 1 (East vs. West) accuracy:", accuracy_score(y_test_semi1, y_pred_semi1))


# ========================================
# Part 2: Semifinal 2 (South vs. Midwest)
# ========================================
# Create binary target: 0 if South wins, 1 if Midwest wins.
df['target_semi2'] = df.apply(
    lambda row: 0 if row['SemifinalWinner_South_Midwest'] == row['RegionWinner_South']
                else (1 if row['SemifinalWinner_South_Midwest'] == row['RegionWinner_Midwest'] else -1),
    axis=1
)
df_semi2 = df[df['target_semi2'] != -1].copy().reset_index(drop=True)

# Define features for Semifinal 2 (adjust as needed)
feature_cols_semi2 = [
    'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
    'InstitutionCity_RegionWinner_East', 'InstitutionState_RegionWinner_East', 'InstitutionConference_RegionWinner_East',
    'InstitutionEnrollment_Male_RegionWinner_East', 'InstitutionEnrollment_Female_RegionWinner_East', 'InstitutionEnrollment_Total_RegionWinner_East',
    'RegularSeasonWins_RegionWinner_East', 'RegularSeasonLosses_RegionWinner_East', 'RegularSeasonAverageAttendance_RegionWinner_East', 'RegularSeasonAverageScore_RegionWinner_East',
    'InstitutionCity_RegionWinner_West', 'InstitutionState_RegionWinner_West', 'InstitutionConference_RegionWinner_West',
    'InstitutionEnrollment_Male_RegionWinner_West', 'InstitutionEnrollment_Female_RegionWinner_West', 'InstitutionEnrollment_Total_RegionWinner_West',
    'RegularSeasonWins_RegionWinner_West', 'RegularSeasonLosses_RegionWinner_West', 'RegularSeasonAverageAttendance_RegionWinner_West', 'RegularSeasonAverageScore_RegionWinner_West',
    'InstitutionCity_RegionWinner_South', 'InstitutionState_RegionWinner_South', 'InstitutionConference_RegionWinner_South',
    'InstitutionEnrollment_Male_RegionWinner_South', 'InstitutionEnrollment_Female_RegionWinner_South', 'InstitutionEnrollment_Total_RegionWinner_South',
    'RegularSeasonWins_RegionWinner_South', 'RegularSeasonLosses_RegionWinner_South', 'RegularSeasonAverageAttendance_RegionWinner_South', 'RegularSeasonAverageScore_RegionWinner_South',
    'InstitutionCity_RegionWinner_Midwest', 'InstitutionState_RegionWinner_Midwest', 'InstitutionConference_RegionWinner_Midwest',
    'InstitutionEnrollment_Male_RegionWinner_Midwest', 'InstitutionEnrollment_Female_RegionWinner_Midwest', 'InstitutionEnrollment_Total_RegionWinner_Midwest',
    'RegularSeasonWins_RegionWinner_Midwest', 'RegularSeasonLosses_RegionWinner_Midwest', 'RegularSeasonAverageAttendance_RegionWinner_Midwest', 'RegularSeasonAverageScore_RegionWinner_Midwest',
    'Distance_RegionWinner_East', 'Distance_RegionWinner_West', 'Distance_RegionWinner_South', 'Distance_RegionWinner_Midwest'
]

X_semi2 = df_semi2[feature_cols_semi2]
y_semi2 = df_semi2['target_semi2']

categorical_cols_semi2 = [
    'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
    'InstitutionCity_RegionWinner_East', 'InstitutionState_RegionWinner_East', 'InstitutionConference_RegionWinner_East',
    'InstitutionCity_RegionWinner_West', 'InstitutionState_RegionWinner_West', 'InstitutionConference_RegionWinner_West',
    'InstitutionCity_RegionWinner_South', 'InstitutionState_RegionWinner_South', 'InstitutionConference_RegionWinner_South',
    'InstitutionCity_RegionWinner_Midwest', 'InstitutionState_RegionWinner_Midwest', 'InstitutionConference_RegionWinner_Midwest'
]
X_semi2 = pd.get_dummies(X_semi2, columns=categorical_cols_semi2, drop_first=True)

# Split data
X_train_semi2, X_test_semi2, y_train_semi2, y_test_semi2 = train_test_split(X_semi2, y_semi2, test_size=0.30, random_state=42)

# Set up GridSearchCV for Semifinal 2
grid_search_semi2 = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search_semi2.fit(X_train_semi2, y_train_semi2)

best_model_semi2 = grid_search_semi2.best_estimator_
print("Semifinal 2 best parameters:", grid_search_semi2.best_params_)

y_pred_semi2 = best_model_semi2.predict(X_test_semi2)
print("Semifinal 2 (South vs. Midwest) accuracy:", accuracy_score(y_test_semi2, y_pred_semi2))


# ========================================
# Part 3: Final (National Champion)
# ========================================
# Create binary target: 0 if National Champion equals Semifinal 1 winner, 1 if it equals Semifinal 2 winner.
df['target_final'] = df.apply(
    lambda row: 0 if row['NationalChampion'] == row['SemifinalWinner_East_West']
                else (1 if row['NationalChampion'] == row['SemifinalWinner_South_Midwest'] else -1),
    axis=1
)
df_final = df[df['target_final'] != -1].copy().reset_index(drop=True)

# Define features for the Final round (using all regional features; adjust as needed)
feature_cols_final = [
    'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
    'InstitutionCity_RegionWinner_East', 'InstitutionState_RegionWinner_East', 'InstitutionConference_RegionWinner_East',
    'InstitutionEnrollment_Male_RegionWinner_East', 'InstitutionEnrollment_Female_RegionWinner_East', 'InstitutionEnrollment_Total_RegionWinner_East',
    'RegularSeasonWins_RegionWinner_East', 'RegularSeasonLosses_RegionWinner_East', 'RegularSeasonAverageAttendance_RegionWinner_East', 'RegularSeasonAverageScore_RegionWinner_East',
    'InstitutionCity_RegionWinner_West', 'InstitutionState_RegionWinner_West', 'InstitutionConference_RegionWinner_West',
    'InstitutionEnrollment_Male_RegionWinner_West', 'InstitutionEnrollment_Female_RegionWinner_West', 'InstitutionEnrollment_Total_RegionWinner_West',
    'RegularSeasonWins_RegionWinner_West', 'RegularSeasonLosses_RegionWinner_West', 'RegularSeasonAverageAttendance_RegionWinner_West', 'RegularSeasonAverageScore_RegionWinner_West',
    'InstitutionCity_RegionWinner_South', 'InstitutionState_RegionWinner_South', 'InstitutionConference_RegionWinner_South',
    'InstitutionEnrollment_Male_RegionWinner_South', 'InstitutionEnrollment_Female_RegionWinner_South', 'InstitutionEnrollment_Total_RegionWinner_South',
    'RegularSeasonWins_RegionWinner_South', 'RegularSeasonLosses_RegionWinner_South', 'RegularSeasonAverageAttendance_RegionWinner_South', 'RegularSeasonAverageScore_RegionWinner_South',
    'InstitutionCity_RegionWinner_Midwest', 'InstitutionState_RegionWinner_Midwest', 'InstitutionConference_RegionWinner_Midwest',
    'InstitutionEnrollment_Male_RegionWinner_Midwest', 'InstitutionEnrollment_Female_RegionWinner_Midwest', 'InstitutionEnrollment_Total_RegionWinner_Midwest',
    'RegularSeasonWins_RegionWinner_Midwest', 'RegularSeasonLosses_RegionWinner_Midwest', 'RegularSeasonAverageAttendance_RegionWinner_Midwest', 'RegularSeasonAverageScore_RegionWinner_Midwest',
    'Distance_RegionWinner_East', 'Distance_RegionWinner_West', 'Distance_RegionWinner_South', 'Distance_RegionWinner_Midwest'
]

X_final = df_final[feature_cols_final]
y_final = df_final['target_final']

categorical_cols_final = [
    'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
    'InstitutionCity_RegionWinner_East', 'InstitutionState_RegionWinner_East', 'InstitutionConference_RegionWinner_East',
    'InstitutionCity_RegionWinner_West', 'InstitutionState_RegionWinner_West', 'InstitutionConference_RegionWinner_West',
    'InstitutionCity_RegionWinner_South', 'InstitutionState_RegionWinner_South', 'InstitutionConference_RegionWinner_South',
    'InstitutionCity_RegionWinner_Midwest', 'InstitutionState_RegionWinner_Midwest', 'InstitutionConference_RegionWinner_Midwest'
]
X_final = pd.get_dummies(X_final, columns=categorical_cols_final, drop_first=True)

# Split data
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X_final, y_final, test_size=0.30, random_state=42)

# Set up GridSearchCV for Final round
grid_search_final = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search_final.fit(X_train_final, y_train_final)

best_model_final = grid_search_final.best_estimator_
print("Final round best parameters:", grid_search_final.best_params_)

y_pred_final = best_model_final.predict(X_test_final)
print("Final round (National Champion) accuracy:", accuracy_score(y_test_final, y_pred_final))

Fitting 3 folds for each of 72 candidates, totalling 216 fits


In [4]:
import pandas as pd
from xgboost import XGBClassifier


test_df = pd.read_csv('/Users/keerthianand/Downloads/bracket_test_allstats_2.csv')
# ================================
# Data Preparation (All Stages)
# ================================
# Convert team ID columns to integer type
test_df['RegionWinner_East'] = test_df['RegionWinner_East'].astype(int)
test_df['RegionWinner_West'] = test_df['RegionWinner_West'].astype(int)
test_df['RegionWinner_South'] = test_df['RegionWinner_South'].astype(int)
test_df['RegionWinner_Midwest'] = test_df['RegionWinner_Midwest'].astype(int)


# ========================================
# Part 1: Semifinal 1 (East vs. West)
# ========================================


# Define features for Semifinal 1 (adjust as needed)
feature_cols_semi1 = [
    'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
    'InstitutionCity_RegionWinner_East', 'InstitutionState_RegionWinner_East', 'InstitutionConference_RegionWinner_East',
    'InstitutionEnrollment_Male_RegionWinner_East', 'InstitutionEnrollment_Female_RegionWinner_East', 'InstitutionEnrollment_Total_RegionWinner_East',
    'RegularSeasonWins_RegionWinner_East', 'RegularSeasonLosses_RegionWinner_East', 'RegularSeasonAverageAttendance_RegionWinner_East', 'RegularSeasonAverageScore_RegionWinner_East',
    'InstitutionCity_RegionWinner_West', 'InstitutionState_RegionWinner_West', 'InstitutionConference_RegionWinner_West',
    'InstitutionEnrollment_Male_RegionWinner_West', 'InstitutionEnrollment_Female_RegionWinner_West', 'InstitutionEnrollment_Total_RegionWinner_West',
    'RegularSeasonWins_RegionWinner_West', 'RegularSeasonLosses_RegionWinner_West', 'RegularSeasonAverageAttendance_RegionWinner_West', 'RegularSeasonAverageScore_RegionWinner_West',
    'InstitutionCity_RegionWinner_South', 'InstitutionState_RegionWinner_South', 'InstitutionConference_RegionWinner_South',
    'InstitutionEnrollment_Male_RegionWinner_South', 'InstitutionEnrollment_Female_RegionWinner_South', 'InstitutionEnrollment_Total_RegionWinner_South',
    'RegularSeasonWins_RegionWinner_South', 'RegularSeasonLosses_RegionWinner_South', 'RegularSeasonAverageAttendance_RegionWinner_South', 'RegularSeasonAverageScore_RegionWinner_South',
    'InstitutionCity_RegionWinner_Midwest', 'InstitutionState_RegionWinner_Midwest', 'InstitutionConference_RegionWinner_Midwest',
    'InstitutionEnrollment_Male_RegionWinner_Midwest', 'InstitutionEnrollment_Female_RegionWinner_Midwest', 'InstitutionEnrollment_Total_RegionWinner_Midwest',
    'RegularSeasonWins_RegionWinner_Midwest', 'RegularSeasonLosses_RegionWinner_Midwest', 'RegularSeasonAverageAttendance_RegionWinner_Midwest', 'RegularSeasonAverageScore_RegionWinner_Midwest',
    'Distance_RegionWinner_East', 'Distance_RegionWinner_West', 'Distance_RegionWinner_South', 'Distance_RegionWinner_Midwest'
]

X_test_semi1 = test_df[feature_cols_semi1]

# List categorical columns for one-hot encoding
categorical_cols_semi1 = [
    'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
    'InstitutionCity_RegionWinner_East', 'InstitutionState_RegionWinner_East', 'InstitutionConference_RegionWinner_East',
    'InstitutionCity_RegionWinner_West', 'InstitutionState_RegionWinner_West', 'InstitutionConference_RegionWinner_West',
    'InstitutionCity_RegionWinner_South', 'InstitutionState_RegionWinner_South', 'InstitutionConference_RegionWinner_South',
    'InstitutionCity_RegionWinner_Midwest', 'InstitutionState_RegionWinner_Midwest', 'InstitutionConference_RegionWinner_Midwest'
]
X_test_semi1 = pd.get_dummies(X_test_semi1, columns=categorical_cols_semi1, drop_first=True)

# Ensure the test data has the same columns as the training data
X_test_semi1 = X_test_semi1.reindex(columns=X_train_semi1.columns, fill_value=0)

# Predict using the best model for Semifinal 1
y_pred_semi1_test = best_model_semi1.predict(X_test_semi1)
print("Semifinal 1 (East vs. West) predictions:", y_pred_semi1_test)

# ========================================
# Part 2: Semifinal 2 (South vs. Midwest)
# ========================================


# Define features for Semifinal 2 (adjust as needed)
feature_cols_semi2 = [
'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
    'InstitutionCity_RegionWinner_East', 'InstitutionState_RegionWinner_East', 'InstitutionConference_RegionWinner_East',
    'InstitutionEnrollment_Male_RegionWinner_East', 'InstitutionEnrollment_Female_RegionWinner_East', 'InstitutionEnrollment_Total_RegionWinner_East',
    'RegularSeasonWins_RegionWinner_East', 'RegularSeasonLosses_RegionWinner_East', 'RegularSeasonAverageAttendance_RegionWinner_East', 'RegularSeasonAverageScore_RegionWinner_East',
    'InstitutionCity_RegionWinner_West', 'InstitutionState_RegionWinner_West', 'InstitutionConference_RegionWinner_West',
    'InstitutionEnrollment_Male_RegionWinner_West', 'InstitutionEnrollment_Female_RegionWinner_West', 'InstitutionEnrollment_Total_RegionWinner_West',
    'RegularSeasonWins_RegionWinner_West', 'RegularSeasonLosses_RegionWinner_West', 'RegularSeasonAverageAttendance_RegionWinner_West', 'RegularSeasonAverageScore_RegionWinner_West',
    'InstitutionCity_RegionWinner_South', 'InstitutionState_RegionWinner_South', 'InstitutionConference_RegionWinner_South',
    'InstitutionEnrollment_Male_RegionWinner_South', 'InstitutionEnrollment_Female_RegionWinner_South', 'InstitutionEnrollment_Total_RegionWinner_South',
    'RegularSeasonWins_RegionWinner_South', 'RegularSeasonLosses_RegionWinner_South', 'RegularSeasonAverageAttendance_RegionWinner_South', 'RegularSeasonAverageScore_RegionWinner_South',
    'InstitutionCity_RegionWinner_Midwest', 'InstitutionState_RegionWinner_Midwest', 'InstitutionConference_RegionWinner_Midwest',
    'InstitutionEnrollment_Male_RegionWinner_Midwest', 'InstitutionEnrollment_Female_RegionWinner_Midwest', 'InstitutionEnrollment_Total_RegionWinner_Midwest',
    'RegularSeasonWins_RegionWinner_Midwest', 'RegularSeasonLosses_RegionWinner_Midwest', 'RegularSeasonAverageAttendance_RegionWinner_Midwest', 'RegularSeasonAverageScore_RegionWinner_Midwest',
    'Distance_RegionWinner_East', 'Distance_RegionWinner_West', 'Distance_RegionWinner_South', 'Distance_RegionWinner_Midwest'
]

X_test_semi2 = test_df[feature_cols_semi2]

categorical_cols_semi2 = [
    'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
    'InstitutionCity_RegionWinner_East', 'InstitutionState_RegionWinner_East', 'InstitutionConference_RegionWinner_East',
    'InstitutionCity_RegionWinner_West', 'InstitutionState_RegionWinner_West', 'InstitutionConference_RegionWinner_West',
    'InstitutionCity_RegionWinner_South', 'InstitutionState_RegionWinner_South', 'InstitutionConference_RegionWinner_South',
    'InstitutionCity_RegionWinner_Midwest', 'InstitutionState_RegionWinner_Midwest', 'InstitutionConference_RegionWinner_Midwest'
]
X_test_semi2 = pd.get_dummies(X_test_semi2, columns=categorical_cols_semi2, drop_first=True)

# Ensure the test data has the same columns as the training data
X_test_semi2 = X_test_semi2.reindex(columns=X_train_semi2.columns, fill_value=0)

# Predict using the best model for Semifinal 2
y_pred_semi2_test = best_model_semi2.predict(X_test_semi2)
print("Semifinal 2 (South vs. Midwest) predictions:", y_pred_semi2_test)

# ========================================
# Part 3: Final (National Champion)
# ========================================

# Define features for the Final round (using all regional features; adjust as needed)
feature_cols_final = [
    'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
    'InstitutionCity_RegionWinner_East', 'InstitutionState_RegionWinner_East', 'InstitutionConference_RegionWinner_East',
    'InstitutionEnrollment_Male_RegionWinner_East', 'InstitutionEnrollment_Female_RegionWinner_East', 'InstitutionEnrollment_Total_RegionWinner_East',
    'RegularSeasonWins_RegionWinner_East', 'RegularSeasonLosses_RegionWinner_East', 'RegularSeasonAverageAttendance_RegionWinner_East', 'RegularSeasonAverageScore_RegionWinner_East',
    'InstitutionCity_RegionWinner_West', 'InstitutionState_RegionWinner_West', 'InstitutionConference_RegionWinner_West',
    'InstitutionEnrollment_Male_RegionWinner_West', 'InstitutionEnrollment_Female_RegionWinner_West', 'InstitutionEnrollment_Total_RegionWinner_West',
    'RegularSeasonWins_RegionWinner_West', 'RegularSeasonLosses_RegionWinner_West', 'RegularSeasonAverageAttendance_RegionWinner_West', 'RegularSeasonAverageScore_RegionWinner_West',
    'InstitutionCity_RegionWinner_South', 'InstitutionState_RegionWinner_South', 'InstitutionConference_RegionWinner_South',
    'InstitutionEnrollment_Male_RegionWinner_South', 'InstitutionEnrollment_Female_RegionWinner_South', 'InstitutionEnrollment_Total_RegionWinner_South',
    'RegularSeasonWins_RegionWinner_South', 'RegularSeasonLosses_RegionWinner_South', 'RegularSeasonAverageAttendance_RegionWinner_South', 'RegularSeasonAverageScore_RegionWinner_South',
    'InstitutionCity_RegionWinner_Midwest', 'InstitutionState_RegionWinner_Midwest', 'InstitutionConference_RegionWinner_Midwest',
    'InstitutionEnrollment_Male_RegionWinner_Midwest', 'InstitutionEnrollment_Female_RegionWinner_Midwest', 'InstitutionEnrollment_Total_RegionWinner_Midwest',
    'RegularSeasonWins_RegionWinner_Midwest', 'RegularSeasonLosses_RegionWinner_Midwest', 'RegularSeasonAverageAttendance_RegionWinner_Midwest', 'RegularSeasonAverageScore_RegionWinner_Midwest',
    'Distance_RegionWinner_East', 'Distance_RegionWinner_West', 'Distance_RegionWinner_South', 'Distance_RegionWinner_Midwest'
]

X_test_final = test_df[feature_cols_final]

categorical_cols_final = [
    'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest',
    'InstitutionCity_RegionWinner_East', 'InstitutionState_RegionWinner_East', 'InstitutionConference_RegionWinner_East',
    'InstitutionCity_RegionWinner_West', 'InstitutionState_RegionWinner_West', 'InstitutionConference_RegionWinner_West',
    'InstitutionCity_RegionWinner_South', 'InstitutionState_RegionWinner_South', 'InstitutionConference_RegionWinner_South',
    'InstitutionCity_RegionWinner_Midwest', 'InstitutionState_RegionWinner_Midwest', 'InstitutionConference_RegionWinner_Midwest'
]
X_test_final = pd.get_dummies(X_test_final, columns=categorical_cols_final, drop_first=True)

# Ensure the test data has the same columns as the training data
X_test_final = X_test_final.reindex(columns=X_train_final.columns, fill_value=0)

# Predict using the best model for Final round
y_pred_final_test = best_model_final.predict(X_test_final)
print("Final round (National Champion) predictions:", y_pred_final_test)

Semifinal 1 (East vs. West) predictions: [1 1 0 ... 1 0 0]
Semifinal 2 (South vs. Midwest) predictions: [0 0 0 ... 0 0 0]
Final round (National Champion) predictions: [0 0 1 ... 1 0 0]


In [6]:
# Part 1: Semifinal 1 (East vs. West)
# ========================================
# Replace predictions with institution IDs
test_df['Semifinal1_Winner'] = test_df.apply(
    lambda row: row['RegionWinner_East'] if y_pred_semi1_test[test_df.index.get_loc(row.name)] == 0
    else row['RegionWinner_West'],
    axis=1
)

# ========================================
# Part 2: Semifinal 2 (South vs. Midwest)
# ========================================
# Replace predictions with institution IDs
test_df['Semifinal2_Winner'] = test_df.apply(
    lambda row: row['RegionWinner_South'] if y_pred_semi2_test[test_df.index.get_loc(row.name)] == 0
    else row['RegionWinner_Midwest'],
    axis=1
)

# ========================================
# Part 3: Final (National Champion)
# ========================================
# Replace predictions with institution IDs
test_df['NationalChampion_Predicted'] = test_df.apply(
    lambda row: row['Semifinal1_Winner'] if y_pred_final_test[test_df.index.get_loc(row.name)] == 0
    else row['Semifinal2_Winner'],
    axis=1
)

# Display the results
print("Semifinal 1 Winners (Institution IDs):", test_df['Semifinal1_Winner'].tolist())
print("Semifinal 2 Winners (Institution IDs):", test_df['Semifinal2_Winner'].tolist())
print("National Champion Predictions (Institution IDs):", test_df['NationalChampion_Predicted'].tolist())



Semifinal 1 Winners (Institution IDs): [457, 457, 311, 164, 164, 164, 164, 164, 29, 29, 311, 164, 164, 29, 29, 164, 164, 164, 164, 164, 164, 164, 164, 430, 301, 164, 29, 164, 164, 457, 37, 164, 164, 29, 311, 29, 301, 164, 311, 457, 301, 164, 29, 164, 29, 37, 311, 29, 51, 37, 457, 29, 164, 301, 164, 29, 164, 164, 164, 164, 29, 164, 164, 311, 164, 416, 457, 37, 164, 311, 9, 37, 164, 164, 37, 164, 164, 164, 164, 457, 164, 164, 457, 29, 457, 164, 164, 164, 29, 311, 416, 301, 29, 164, 164, 311, 457, 164, 37, 311, 164, 164, 311, 311, 311, 164, 29, 164, 29, 164, 311, 164, 164, 457, 164, 164, 164, 29, 37, 164, 311, 311, 164, 164, 29, 164, 164, 311, 164, 164, 51, 164, 164, 457, 164, 457, 311, 164, 164, 164, 164, 457, 164, 311, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 164, 457, 29, 301, 164, 164, 164, 457, 301, 164, 164, 311, 164, 164, 37, 8, 164, 164, 164, 164, 164, 164, 164, 164, 311, 164, 457, 457, 164, 311, 164, 29, 678, 457, 164, 311, 164, 457, 29, 457, 164, 164, 164, 164, 164, 164

In [7]:
test_df.to_csv('/Users/keerthianand/Downloads/bracket_test_predicted_stats_5.csv', index=False)